In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#pd.set_option('display.max_columns', None)

import plotly.express as px #for visualization
import matplotlib.pyplot as plt #for visualization 

#df = pd.read_excel(r'/content/drive/My Drive/ChurnPrediction/wtw2112 faa.xlsx')

df=pd.read_csv(r'/content/drive/My Drive/ChurnPrediction/newface.csv',sep=";",encoding='ISO-8859-1')
df.head()
df.info(verbose=True,show_counts=True)


In [3]:
newdf=df.copy()

In [4]:
newdf=newdf.drop(['Discount_pol_start','Insurance_tenure','All_LoB_tenure','Motor_contracts_renewal','Residence_municipality','N_changes_main_cov_ref_period','Policyholder_age','Amount_last_period_CCO'], axis=1)

Discretization (Convert float to interval columns). I set number of bins to 5. But you can change it to 3,7,10 to test  ,model accuracy. It is hyper parameter you can tune it.

In [ ]:
from pandas import DataFrame

interval_column=["dif_with_disc_MAIN","dif_with_disc_ALL","Amount_last_period",
                 "Amount_last_period_TPL_Bod","Amount_last_period_TPL_Mat","Amount_last_period_Glass",
                 "Amount_last_3y","Amount_last_3y_TPL_Bod","Amount_last_3y_TPL_Mat","Amount_last_3y_CCO",
                 "Amount_last_claim_ref_period","days_until_last_claim",
                 "Amount_paid_current_period","Motor_tenure_with_OD","Contracts_lapsed_ref_period"]

for col in interval_column:
  print("===================> Discretization for column of ",col)
  data=df[col]
  #Discretization
  interval_data=pd.qcut(data, 5,duplicates='drop')
  dataset = DataFrame(interval_data)
  print("Distributation of data",dataset.value_counts())
  print("number of null values:",dataset.isna().sum())
  # fill na 
  dataset[col] = dataset[col].cat.add_categories('No claims')
  dataset[col]=dataset[col].fillna("No claims")
  print("number of null values:",dataset.isna().sum())
  print("Distributation of data")
  print(dataset.value_counts())
  newdf[col]=dataset[col]
newdf.info(verbose=True,show_counts=True)

Filling categorical values with Average of colu mn values

In [ ]:
catgorical_columns=["Residence_district","Plate_origin","Type_of_last_claim_ref_period"]
for col in catgorical_columns:
  newdf[col]=newdf[col].fillna("NA")

newdf.info(verbose=True,show_counts=True)

Filling numerical values with Average of column values




In [ ]:
numerical_columns=['Policyholder_license_age','Driver_age','Driver_license_age']

for col in numerical_columns:
  newdf[col]=newdf[col].fillna(df[col].mean()).round(0)

newdf.info(verbose=True,show_counts=True)

Filling numerical values with 0

In [ ]:
zer0_numerical_columns=['N_claims_last_period_Glass','N_claims_last_period','N_claims_last_period_TPL_Bod','N_claims_last_period_TPL_Mat','N_claims_last_period_CCO','N_claims_last_period_CCO','N_claims_last_period_Glass','N_claims_last_3y','N_claims_last_3y_TPL_Mat','N_claims_last_3y_TPL_Bod','N_claims_last_3y_CCO','N_paid_current_period','N_paid_current_period']

for col in zer0_numerical_columns:
  newdf[col]=newdf[col].fillna(0)

newdf.info(verbose=True,show_counts=True)

Removing some columns we will not need in model


In [ ]:
newdf.info(verbose=True,show_counts=True)

"Pack","Residence_country"

Filling NA in  object types  with Mode

In [ ]:

newdf.fillna(newdf.select_dtypes(include='object').mode().iloc[0], inplace=True)
newdf.info(verbose=True,show_counts=True)

Lebel encoding for categorical columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
for col in newdf.columns[0:]:
     if newdf[col].dtype == 'object':
             print(col)
             newdf[col] = lb_make.fit_transform(newdf[col].astype(str))
newdf.info(verbose=True,show_counts=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
for col in newdf.columns[0:]:
     if newdf[col].dtype == 'category':
             print(col)
             newdf[col] = lb_make.fit_transform(newdf[col].astype(str))
newdf.info(verbose=True,show_counts=True)

In [ ]:
newdf.to_csv('Cleaned_Dataset.csv', index=False)

Correlation Matrix

In [ ]:
import seaborn as sn
#Set and compute the Correlation Matrix:
sn.set(style="white")
corr = newdf.corr()
#Generate a mask for the upper triangle:
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
#Set up the matplotlib figure and a diverging colormap:
f, ax = plt.subplots(figsize=(18, 15))
cmap = sn.diverging_palette(220, 10, as_cmap=True)
#Draw the heatmap with the mask and correct aspect ratio:
sn.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
square=True, linewidths=.5, cbar_kws={"shrink": .5})

Split Dataset to tran and test

In [13]:
#Identify response variable:
    
response = newdf["RENEWAL_POLICY"]
dataset = newdf.drop(columns="RENEWAL_POLICY")

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset,response, test_size = 0.2,random_state = 0)
#to resolve any class imbalance - use stratify parameter.
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

Number transactions X_train dataset:  (214010, 68)
Number transactions y_train dataset:  (214010,)
Number transactions X_test dataset:  (53503, 68)
Number transactions y_test dataset:  (53503,)


Random Forest


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score

rfc = RandomForestClassifier(class_weight = "balanced")

# fit the predictor and target
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)
# predict
print('Accuracy score:',accuracy_score(y_test, rfc_predict))
cm = confusion_matrix(y_test, rfc_predict)
print(cm)
#print('F1 score:',f1_score(y_test, rfc_predict))

Accuracy score: 0.9861689998691662
[[ 3464   515]
 [  225 49299]]


Ensemble

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
# importing voting classifier
from sklearn.ensemble import VotingClassifier
model_1 = LogisticRegression()
model_2 = XGBClassifier()
model_3 = RandomForestClassifier()
# Making the final model using voting classifier
final_model = VotingClassifier(
    estimators=[('lr', model_1), ('xgb', model_2), ('rf', model_3)], voting='hard')
 
# training all the model on the train dataset
final_model.fit(X_train, y_train)
 
# predicting the output on the test dataset
pred_final = final_model.predict(X_test)
 

# predict
print('Accuracy score:',accuracy_score(y_test, pred_final))
cm = confusion_matrix(y_test, pred_final)
print(cm)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy score: 0.9862624525727529
[[ 3442   537]
 [  198 49326]]


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score
# Training the Model
logreg = LogisticRegression(C=1e5)
logreg.fit(X_train, y_train)
pred_final=logreg.predict(X_test)
# predict
print('Accuracy score:',accuracy_score(y_test, pred_final))
cm = confusion_matrix(y_test, pred_final)
print(cm)



Accuracy score: 0.9247518830719773
[[   70  3909]
 [  117 49407]]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
model_2 = xgb.XGBClassifier()
# training all the model on the training dataset

model_2.fit(X_train, y_train)

 
# predicting the output on the validation dataset

pred_2 = model_2.predict(X_test)
# predict
print('Accuracy score:',accuracy_score(y_test, pred_2))
cm = confusion_matrix(y_test, pred_2)
print(cm)

Accuracy score: 0.9865988823056651
[[ 3492   487]
 [  230 49294]]


Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)
# predicting the output on the validation dataset

# predict
print('Accuracy score:',accuracy_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)

Accuracy score: 0.9763938470739959
[[ 3415   564]
 [  699 48825]]


KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 10, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test, y_pred)
ac = accuracy_score(y_test,y_pred)
print(ac)

0.9251070033456068


In [ ]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE(k_neighbors=5)
X_smote, y_smote = oversample.fit_resample(X_train, y_train)

X_train, y_train = X_smote, y_smote
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)

Checking Under Sampling

In [ ]:
# import library
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42, replacement=True)# fit predictor and target variable
x_rus, y_rus = rus.fit_resample(X_train, y_train)

print('original dataset shape:',x_rus.shape)
print('Resample dataset shape', y_rus.shape)

original dataset shape: (27932, 125)
Resample dataset shape (27932,)


In [ ]:
print('original dataset shape:',x_rus.shape)
print('Resample dataset shape', y_rus.shape)

original dataset shape: (27856, 125)
Resample dataset shape (27856,)


Checking OverSampling 

In [ ]:
#Importing essential libraries
import matplotlib.pyplot as plt
from statistics import mean
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier

#Build SRF model
SRF = RandomForestClassifier(n_estimators=150, random_state=0)
#Create Stratified K-fold cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scoring = ('f1', 'recall', 'precision')
#Evaluate SRF model
scores = cross_validate(SRF, dataset, response, scoring=scoring, cv=cv)
#Get average evaluation metrics
print('Mean f1: %.3f' % mean(scores['test_f1']))
print('Mean recall: %.3f' % mean(scores['test_recall']))
print('Mean precision: %.3f' % mean(scores['test_precision']))

#Randomly spilt dataset to test and train set
X_train, X_test, y_train, y_test = train_test_split(dataset, response, test_size=0.1, stratify=response)
#Train SRF
SRF.fit(X_train, y_train)
#SRF prediction result
y_pred = SRF.predict(X_test)
#Create confusion matrix
fig = plot_confusion_matrix(SRF, X_test, y_test, display_labels=['Will Not churn', 'Will churn'], cmap='Greens')
plt.title('Standard Random Forest Confusion Matrix')
plt.show()

In [ ]:
#Importing essential libraries
import matplotlib.pyplot as plt
from statistics import mean
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

#Use SMOTE to oversample the minority class
oversample = SMOTE()
over_X, over_y = oversample.fit_resample(dataset, response)
over_X_train, over_X_test, over_y_train, over_y_test = train_test_split(over_X, over_y, test_size=0.1, stratify=over_y)
#Build SMOTE SRF model
SMOTE_SRF = RandomForestClassifier()
#Create Stratified K-fold cross validation
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
scoring = ('f1', 'recall', 'precision')
#Evaluate SMOTE SRF model
scores = cross_validate(SMOTE_SRF, over_X, over_y, scoring=scoring, cv=cv)
#Get average evaluation metrics
print('Mean f1: %.3f' % mean(scores['test_f1']))
print('Mean recall: %.3f' % mean(scores['test_recall']))
print('Mean precision: %.3f' % mean(scores['test_precision']))

#Randomly spilt dataset to test and train set
X_train, X_test, y_train, y_test = train_test_split(dataset, response, test_size=0.1, stratify=response)
#Train SMOTE SRF
SMOTE_SRF.fit(over_X_train, over_y_train)
#SMOTE SRF prediction result
y_pred = SMOTE_SRF.predict(X_test)
#Create confusion matrix
fig = plot_confusion_matrix(SMOTE_SRF, X_test, y_test, display_labels=['Will Not Buy', 'Will Buy'], cmap='Greens')
plt.title('SMOTE + Standard Random Forest Confusion Matrix')
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train2 = pd.DataFrame(sc_X.fit_transform(X_train))
X_train2.columns = X_train.columns.values
X_train2.index = X_train.index.values
X_train = X_train2
X_test2 = pd.DataFrame(sc_X.transform(X_test))
X_test2.columns = X_test.columns.values
X_test2.index = X_test.index.values
X_test = X_test2